In [2]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [1]:
!pip install  xgboost

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 166.7 MB 10 kB/s /s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p37/bin/python -m pip install --upgrade pip' command.


In [48]:
import pandas as pd
import numpy as np
from datetime import datetime
import holidays
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from scipy.stats import  randint
from scipy.stats import uniform
import math

## Build X_train

In [7]:
X_train = pd.read_csv('fl01_train')

In [30]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990398 entries, 0 to 991893
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   distance          990398 non-null  float64
 1   uni_fl            990398 non-null  float64
 2   arr_airport_hour  990398 non-null  float64
 3   dep_airport_hour  990398 non-null  float64
 4   is_holiday        990398 non-null  int64  
 5   dow_sin           990398 non-null  float64
 6   dow_cos           990398 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 60.4 MB


In [9]:
X_train = X_train[['fl_date','op_unique_carrier','op_carrier_fl_num','origin_airport_id','crs_dep_time','dest_airport_id','crs_arr_time','distance','arr_delay']]

In [31]:
X_train.head()

,distance,uni_fl,arr_airport_hour,dep_airport_hour,is_holiday,dow_sin,dow_cos
0,2106.0,-8.642857,-6.284264,0.133843,0,-0.974927,-0.222525
1,1243.0,-1.666667,-4.724080,8.300518,0,-0.781834,0.623486
2,612.0,-1.461538,10.078550,15.230415,0,-0.000005,1.000000
3,515.0,-5.575758,1.696279,4.401942,0,-0.974927,-0.222525
4,1561.0,-7.333333,2.227273,-0.149701,0,-0.000005,1.000000


In [11]:
# after test data, arr_delay should be 0. 
X_train['arr_delay'].fillna(0,inplace=True)

#remove outliner
X_train = X_train[(X_train['arr_delay'] > -80) & (X_train['arr_delay'] <= 480)] 

#combine features
def combine_str_num(x,y):
    return x + str(y)
X_train['uni_fl'] = np.vectorize(combine_str_num)(X_train['op_unique_carrier'],X_train['op_carrier_fl_num'])
X_train.drop(columns=['op_unique_carrier','op_carrier_fl_num'],inplace=True)

# feature coding
unifl_target_mean = X_train.groupby(['uni_fl'])['arr_delay'].mean()
X_train['uni_fl'] =X_train['uni_fl'].map(unifl_target_mean)

In [13]:
## Combine the airportid and related hour
def airport_hour(id,t):
    '''
    combine airport and hour to a new field
    '''
    h = str(t)[:-2]
    if len(h) < 1:
        return str(id) + '24'
    elif len(h) < 2:
        return str(id) + '0' + h
    else:
        return str(id) + h
    
X_train['arr_airport_hour'] = np.vectorize(airport_hour)(X_train['dest_airport_id'],X_train['crs_arr_time'])
X_train['dep_airport_hour'] = np.vectorize(airport_hour)(X_train['origin_airport_id'],X_train['crs_dep_time'])

# feature coding
dep_hour_target_mean = X_train.groupby(['dep_airport_hour'])['arr_delay'].mean()
X_train['dep_airport_hour'] =X_train['dep_airport_hour'].map(dep_hour_target_mean)
arr_hour_target_mean = X_train.groupby(['arr_airport_hour'])['arr_delay'].mean()
X_train['arr_airport_hour'] =X_train['arr_airport_hour'].map(arr_hour_target_mean)

X_train.drop(columns=['dest_airport_id','crs_arr_time','origin_airport_id','crs_dep_time'],inplace=True)

In [16]:
#transform fl_date
#def getmonth(x):
#    return x.split(sep='-')[1]
#X_train['month'] = X_train['fl_date'].apply(getmonth)

def getdayofweek(x):
    year_s, mon_s, day_s = x.split('-')
    fl_d = datetime(int(year_s), int(mon_s), int(day_s))
    return fl_d.weekday() + 1
X_train['day_of_week'] = X_train['fl_date'].apply(getdayofweek)

us_holidays = holidays.UnitedStates()
def isholiday(x):
    year_s, mon_s, day_s = x.split('-')
    if datetime(int(year_s), int(mon_s), int(day_s)) in us_holidays:
        return 1
    else:
        return 0
X_train['is_holiday'] = X_train['fl_date'].apply(isholiday)


pi=3.14159
def transformation(column):
  max_value = column.max()
  sin_values = [math.sin((2*pi*x)/max_value) for x in list(column)]
  cos_values = [math.cos((2*pi*x)/max_value) for x in list(column)]
  return sin_values, cos_values

sin_l, cos_l = transformation(X_train['day_of_week'])

X_train['dow_sin'] = sin_l
X_train['dow_cos'] = cos_l

In [29]:
y_train = X_train['arr_delay']

X_train = X_train.drop(columns=['fl_date','arr_delay','day_of_week'])

#X_train = pd.get_dummies(X_train,columns=['month','day_of_week'])

In [18]:
#scaler = StandardScaler()
#X = scaler.fit_transform(X_train)

In [ ]:
#x_small = X[:100000]
#y_small = y_train[:100000]

### Build X_test

In [67]:
X_test = pd.read_csv('fl01_test')

In [73]:
X_test.head()

,Unnamed: 0,Unnamed: 0.1,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_delay,...,cancellation_code,crs_elapsed_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,month,pred
0,4487605,4487605,2018-01-21,AA,N102NN,9,12478,14771,700,-9.0,...,NaN,409.0,2586.0,NaN,NaN,NaN,NaN,NaN,1,-10.617328
1,4318176,4318176,2018-01-12,WN,N946WN,1832,11278,13204,600,81.0,...,NaN,145.0,759.0,81.0,0.0,1.0,0.0,0.0,1,-4.975438
2,13918767,13918767,2019-01-27,WN,N780SW,1699,14893,10423,825,-3.0,...,NaN,200.0,1481.0,NaN,NaN,NaN,NaN,NaN,1,-1.330982
3,4096990,4096990,2018-01-01,EV,N13992,3978,13930,14524,715,13.0,...,NaN,123.0,642.0,NaN,NaN,NaN,NaN,NaN,1,1.030587
4,4505704,4505704,2018-01-22,F9,N232FR,649,12953,11193,920,-8.0,...,NaN,133.0,585.0,0.0,0.0,19.0,0.0,0.0,1,16.868534


In [77]:
X_test[['pred','arr_delay']].describe()

,pred,arr_delay
count,247974.000000,243605.000000
mean,2.684310,4.038053
std,12.404495,50.270705
min,-57.346992,-244.000000
25%,-5.063082,-16.000000
50%,0.407817,-7.000000
75%,7.765238,7.000000
max,237.414001,1778.000000


In [34]:
X_test = X_test[['fl_date','op_unique_carrier','op_carrier_fl_num','origin_airport_id','crs_dep_time','dest_airport_id','crs_arr_time','distance','arr_delay']]

In [35]:
# after test data, arr_delay should be 0. 
X_test['arr_delay'].fillna(0,inplace=True)

#remove outliner
#X_train = X_train[(X_train['arr_delay'] > -80) & (X_train['arr_delay'] < 180)] 

#combine features
def combine_str_num(x,y):
    return x + str(y)
X_test['uni_fl'] = np.vectorize(combine_str_num)(X_test['op_unique_carrier'],X_test['op_carrier_fl_num'])
X_test.drop(columns=['op_unique_carrier','op_carrier_fl_num'],inplace=True)

# feature coding
#unifl_target_mean = X_test.groupby(['uni_fl'])['arr_delay'].mean()
X_test['uni_fl'] =X_test['uni_fl'].map(unifl_target_mean)

In [37]:
## Combine the airportid and related hour
def airport_hour(id,t):
    '''
    combine airport and hour to a new field
    '''
    h = str(t)[:-2]
    if len(h) < 1:
        return str(id) + '24'
    elif len(h) < 2:
        return str(id) + '0' + h
    else:
        return str(id) + h
    
X_test['arr_airport_hour'] = np.vectorize(airport_hour)(X_test['dest_airport_id'],X_test['crs_arr_time'])
X_test['dep_airport_hour'] = np.vectorize(airport_hour)(X_test['origin_airport_id'],X_test['crs_dep_time'])

# feature coding
#dep_hour_target_mean = X_train.groupby(['dep_airport_hour'])['arr_delay'].mean()
X_test['dep_airport_hour'] =X_test['dep_airport_hour'].map(dep_hour_target_mean)
#arr_hour_target_mean = X_train.groupby(['arr_airport_hour'])['arr_delay'].mean()
X_test['arr_airport_hour'] =X_test['arr_airport_hour'].map(arr_hour_target_mean)

X_test.drop(columns=['dest_airport_id','crs_arr_time','origin_airport_id','crs_dep_time'],inplace=True)

In [39]:
#transform fl_date
#def getmonth(x):
#    return x.split(sep='-')[1]
#X_test['month'] = X_test['fl_date'].apply(getmonth)

def getdayofweek(x):
    year_s, mon_s, day_s = x.split('-')
    fl_d = datetime(int(year_s), int(mon_s), int(day_s))
    return fl_d.weekday() + 1
X_test['day_of_week'] = X_test['fl_date'].apply(getdayofweek)

us_holidays = holidays.UnitedStates()
def isholiday(x):
    year_s, mon_s, day_s = x.split('-')
    if datetime(int(year_s), int(mon_s), int(day_s)) in us_holidays:
        return 1
    else:
        return 0
X_test['is_holiday'] = X_test['fl_date'].apply(isholiday)



pi=3.14159
def transformation(column):
  max_value = column.max()
  sin_values = [math.sin((2*pi*x)/max_value) for x in list(column)]
  cos_values = [math.cos((2*pi*x)/max_value) for x in list(column)]
  return sin_values, cos_values

sin_l, cos_l = transformation(X_test['day_of_week'])

X_test['dow_sin'] = sin_l
X_test['dow_cos'] = cos_l



In [41]:
y_test = X_test['arr_delay']
X_test = X_test.drop(columns=['fl_date','arr_delay','day_of_week'])

#X_test = pd.get_dummies(X_test,columns=['month','day_of_week'])

#X_test = scaler.transform(X_test)

In [14]:
###  XGBoost Modeling

In [49]:
## build benchmark
mean_train = np.mean(y_train)
# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train
# Compute MAE
mae_baseline = mean_absolute_error(y_test, baseline_predictions)
print("Baseline MAE is {:.2f}".format(mae_baseline))

Baseline MAE is 23.82


In [50]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [51]:
params = {
    # Parameters that we are going to tune.
    'max_depth':11,
    'min_child_weight': 5,
    'eta':.3,
    'subsample': 0.7,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
}

In [59]:
params

{'max_depth': 10,
 'min_child_weight': 8,
 'eta': 0.3,
 'subsample': 1.0,
 'colsample_bytree': 1.0,
 'objective': 'reg:squarederror'}

In [52]:
num_boost_round = 30

In [62]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[0]	Test-rmse:49.41376
[1]	Test-rmse:49.18554
[2]	Test-rmse:49.10697
[3]	Test-rmse:49.09246
[4]	Test-rmse:49.09764
[5]	Test-rmse:49.11550
[6]	Test-rmse:49.13012
[7]	Test-rmse:49.14700
[8]	Test-rmse:49.16047
[9]	Test-rmse:49.16863
[10]	Test-rmse:49.17392
[11]	Test-rmse:49.17789
[12]	Test-rmse:49.17870


In [63]:
y_pred = model.predict(dtest)

In [65]:
len(y_pred)

247974

In [64]:
## four round  48.09246
r2_score(y_test,y_pred)

0.025932099879509285

In [94]:
## third round  48.67
r2_score(y_test,y_pred)

0.03500066824855497

In [21]:
## second round with arr_delay <480
#r2_score(y_test, y_pred)

0.02144653066658464

In [27]:
## first round with arr_delay < 180
#

0.013895135285908466

In [53]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(6,11)]
    for colsample in [i/10. for i in range(8,11)]
]

In [54]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=3
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
	MAE 21.380279199999997 for 1 rounds
CV with subsample=1.0, colsample=0.9
	MAE 21.4046162 for 1 rounds
CV with subsample=1.0, colsample=0.8
	MAE 21.444791400000003 for 1 rounds
CV with subsample=0.9, colsample=1.0
	MAE 21.3905212 for 1 rounds
CV with subsample=0.9, colsample=0.9
	MAE 21.4491734 for 1 rounds
CV with subsample=0.9, colsample=0.8
	MAE 21.4839048 for 1 rounds
CV with subsample=0.8, colsample=1.0
	MAE 21.3959534 for 1 rounds
CV with subsample=0.8, colsample=0.9
	MAE 21.4521944 for 1 rounds
CV with subsample=0.8, colsample=0.8
	MAE 21.4824892 for 1 rounds
CV with subsample=0.7, colsample=1.0
	MAE 21.3997034 for 1 rounds
CV with subsample=0.7, colsample=0.9
	MAE 21.461330800000002 for 1 rounds
CV with subsample=0.7, colsample=0.8
	MAE 21.489507600000003 for 0 rounds
CV with subsample=0.6, colsample=1.0
	MAE 21.402399600000003 for 1 rounds
CV with subsample=0.6, colsample=0.9
	MAE 21.466983999999997 for 1 rounds
CV with subsample=0.6, colsa

In [55]:
params['subsample'] = 1.
params['colsample_bytree'] = 1.

In [56]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(4,13,2)
    for min_child_weight in range(2,10,2)
]

In [57]:
# Define initial best params and MAE
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=3
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=4, min_child_weight=2
	MAE 21.481610999999997 for 1 rounds
CV with max_depth=4, min_child_weight=4
	MAE 21.4816116 for 1 rounds
CV with max_depth=4, min_child_weight=6
	MAE 21.481611800000003 for 1 rounds
CV with max_depth=4, min_child_weight=8
	MAE 21.4816118 for 1 rounds
CV with max_depth=6, min_child_weight=2
	MAE 21.4182596 for 1 rounds
CV with max_depth=6, min_child_weight=4
	MAE 21.418219 for 1 rounds
CV with max_depth=6, min_child_weight=6
	MAE 21.4181666 for 1 rounds
CV with max_depth=6, min_child_weight=8
	MAE 21.418130400000003 for 1 rounds
CV with max_depth=8, min_child_weight=2
	MAE 21.384932999999997 for 1 rounds
CV with max_depth=8, min_child_weight=4
	MAE 21.3844036 for 1 rounds
CV with max_depth=8, min_child_weight=6
	MAE 21.384256 for 1 rounds
CV with max_depth=8, min_child_weight=8
	MAE 21.3846292 for 1 rounds
CV with max_depth=10, min_child_weight=2
	MAE 21.378034800000002 for 1 rounds
CV with max_depth=10, min_child_weight=4
	MAE 21.3766188 for 1 r

In [58]:
params['max_depth'] = 10
params['min_child_weight'] = 8

In [ ]:
## first try on XGBoost 
#r2_score(y_test,y_pred)

-0.0005833909885790689

In [ ]:
randm_src = RandomizedSearchCV(estimator=model, param_distributions = parameters,
                               cv = 2, n_iter = 10, n_jobs=-1)